# From Dr Dennett's Guide to Spatial Interaction Modelling 
Code translated to Python by Philip Wilkinson



In [1]:
#import the necessary libraries 
import pandas as pd
import matplotlib.pyplot as plt
#import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import scipy.stats
import numpy as np
from math import sqrt
import statsmodels.formula.api as smf

In [2]:
#set up the metric calculations
def CalcRSqaured(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2
    
    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""
    
    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)
    
    return RMSE

In [5]:

cdatasub1 = pd.read_csv("for_luti/mobility_withoutloop_forLUTI_withname_c_6poi.csv", index_col=0)

In [6]:
cdatasub1

,repaired_avg_time,Dest,Orig,number,dis,avg_time,ave_spd,population,job,log_population,...,O_name,D_name,contiguity,d_finance,d_traffic,d_firm,d_edu,d_residence,d_public,originsimfittedwithC_f_t_f
0,23.635000,3,2,2.0,1.12,23.635000,0.047387,2614,1959,7.868637,...,人民桥,红岭,0,20,62,15,39,26,5,0.0
1,3.966250,4,2,8.0,1.12,3.966250,0.282383,2614,6432,7.868637,...,人民桥,红南,0,46,123,51,52,46,8,26.0
2,4.013636,6,2,22.0,1.12,4.013636,0.279049,2614,27123,7.868637,...,人民桥,滨苑,1,17,97,11,19,30,9,263.0
3,8.991667,8,2,12.0,1.00,8.991667,0.111214,2614,14639,7.868637,...,人民桥,桂园老围,0,153,108,76,34,47,22,16.0
4,7.881379,9,2,29.0,1.41,7.881379,0.178903,2614,7706,7.868637,...,人民桥,松园,0,95,210,312,163,80,24,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332347,58.044464,278,780,0.0,0.00,0.000000,0.000000,4500,710,8.411833,...,志盛,南海玫瑰园,0,3,8,2,10,7,0,0.0
332348,57.226406,272,780,0.0,0.00,0.000000,0.000000,4500,4376,8.411833,...,志盛,渔二,0,4,43,20,11,28,8,1.0
332349,44.529321,663,780,0.0,0.00,0.000000,0.000000,4500,424,8.411833,...,志盛,西涌,0,4,42,2,6,6,9,0.0
332350,54.134134,282,780,0.0,0.00,0.000000,0.000000,4500,11273,8.411833,...,志盛,赤湾,0,10,142,44,10,37,5,2.0


In [7]:
#show the actual flows between boroughs
cdatasubmat1 = pd.pivot_table(cdatasub1, values ="number", index="O_name", columns = "D_name",
                            aggfunc=np.sum, margins=True)
#show the data
cdatasubmat1

D_name,万丰,万科城,三合,三围,三溪,三祝里,三联,上下屋,上合,上坑,...,龙新,龙湖,龙田,龙眼山,龙红格,龙联,龙胜,龙西,龙辉,All
O_name,,,,,,,,,,,,,,,,,,,,,
万丰,NaN,21.0,1.0,18.0,0.0,0.0,1.0,0.0,82.0,0.0,...,0.0,6.0,0.0,0.0,0.0,38.0,5.0,2.0,26.0,32344.0
万科城,0.0,NaN,1.0,1.0,2.0,5.0,170.0,2.0,76.0,690.0,...,12.0,20.0,3.0,0.0,0.0,36.0,193.0,0.0,0.0,60760.0
三合,0.0,82.0,NaN,1.0,0.0,10.0,0.0,4.0,2.0,20.0,...,8.0,1.0,1.0,0.0,0.0,1.0,299.0,0.0,1.0,6525.0
三围,23.0,27.0,0.0,NaN,0.0,0.0,1.0,0.0,162.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,50.0,16077.0
三溪,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,2564.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
龙联,8.0,211.0,0.0,7.0,0.0,0.0,0.0,0.0,98.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,23.0,0.0,132.0,21875.0
龙胜,0.0,880.0,137.0,2.0,0.0,10.0,0.0,8.0,90.0,39.0,...,0.0,30.0,0.0,0.0,0.0,22.0,NaN,1.0,30.0,35147.0
龙西,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,68.0,4.0,8.0,0.0,16.0,0.0,1.0,NaN,0.0,5004.0


In [11]:
cdatasub.columns

Index(['repaired_avg_time', 'Dest', 'Orig', 'number', 'dis', 'avg_time',
       'ave_spd', 'population', 'job', 'log_population', 'log_job',
       'log_repaired_avg_time', 'doubsimfitted', 'O_name', 'D_name', 'O_name1',
       'D_name1'],
      dtype='object')

# Origin- constrained

In [ ]:
#create the formula (the "-1" indicates no intercept in the regression model).
dbl_form = 'number ~ O_name + log_job + log_repaired_avg_time -1'
#run a doubly constrained sim
doubSim = smf.glm(formula = dbl_form, data=cdatasub1, family=sm.families.Poisson()).fit()



In [ ]:
# contiguity -1

In [26]:
#let's have a look at it's summary
doubSim.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 number   No. Observations:               119808
Model:                            GLM   Df Residuals:                   119598
Model Family:                 Poisson   Df Model:                          209
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.5999e+06
Date:                Mon, 31 Jul 2023   Deviance:                   3.0320e+06
Time:                        15:27:43   Pearson chi2:                 1.48e+07
No. Iterations:                    11   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
O_name[三溪]               -0.0486      0.020     -2.378      0.017      -0.089      -0.009
O_name[三联]                0.0722      0.012      6.203      0.000       0.049       0.095
O_name[上坑]               -0.9557      0.015    -65.056      0.000      -0.985      -0.927
O_name[上岭排]              -7.7754      0.180    -43.265      0.000      -8.128      -7.423
O_name[上排]                0.1796      0.012     15.492      0.000       0.157       0.202
O_name[上木古]              -2.0842      0.023    -89.425      0.000      -2.130      -2.039
O_name[上村]               -0.0009      0.014     -0.067      0.946      -0.028       0.026
O_name[下岭排]              -1.6660      0.022    -75.480      0.000      -1.709      -1.623
O_name[下排]               -3.2303      0.024   -135.276      0.000      -3.277      -3.184
O_name[下李朗]               0.4453      0.011     41.663      0.000       0.424       0.466
O_name[下村]                1.6950      0.008    199.906      0.000       1.678       1.712
O_name[下横朗]              -0.3140      0.014    -21.901      0.000      -0.342      -0.286
O_name[下湖]                0.4516      0.011     42.457      0.000       0.431       0.472
O_name[东山]               -0.8556      0.050    -17.174      0.000      -0.953      -0.758
O_name[东方半岛]              0.2747      0.010     26.282      0.000       0.254       0.295
O_name[东方社区二]             0.8510      0.010     83.197      0.000       0.831       0.871
O_name[东海岸]              -0.6479      0.020    -33.078      0.000      -0.686      -0.610
O_name[东涌]               -2.5088      0.131    -19.085      0.000      -2.766      -2.251
O_name[书院街]              -0.0648      0.010     -6.568      0.000      -0.084      -0.045
O_name[五湾]               -1.5151      0.019    -77.809      0.000      -1.553      -1.477
O_name[五联]                1.3996      0.010    139.535      0.000       1.380       1.419
O_name[仙湖]               -1.3897      0.022    -62.680      0.000      -1.433      -1.346
O_name[保安]                0.6060      0.011     53.505      0.000       0.584       0.628
O_name[倚山]               -0.9493      0.021    -45.190      0.000      -0.990      -0.908
O_name[元芬]               -1.4809      0.018    -81.968      0.000      -1.516      -1.445
O_name[光明]               -6.5749      0.243    -27.101      0.000      -7.050      -6.099
O_name[光明凤凰]             -4.6786      0.086    -54.447      0.000      -4.847      -4.510
O_name[公明红星]             -2.5666      0.033    -77.264      0.000      -2.632      -2.501
O_name[兰花]                0.0587      0.010      5.758      0.000       0.039       0.079
O_name[凤凰岗]              -3.2385      0.027   -118.006      0.000      -3.292      -3.185
O_name[南坑]                1.4078      0.008    184.319      0.000       1.393       1.423
O_name[南头城]            

And the various flows and goodness-of-fit statistics?

In [27]:
#get the estimates
cdatasub1["originsimfitted"] = np.round(doubSim.mu)
#here's the matrix
cdatasubmat = cdatasub1.pivot_table(values ="originsimfitted", index="O_name", columns = "D_name",
                                    aggfunc=np.sum, margins=True)
cdatasubmat

D_name,万丰,万科城,三合,三围,三溪,三祝里,三联,上下屋,上合,上坑,...,龙新,龙湖,龙田,龙眼山,龙红格,龙联,龙胜,龙西,龙辉,All
O_name,,,,,,,,,,,,,,,,,,,,,
三溪,2.0,42.0,0.0,0.0,NaN,0.0,1.0,0.0,2.0,1.0,...,7.0,0.0,30.0,0.0,1.0,0.0,1.0,1.0,1.0,2533.0
三联,16.0,420.0,1.0,1.0,1.0,1.0,NaN,0.0,4.0,23.0,...,1.0,8.0,7.0,0.0,18.0,4.0,12.0,0.0,6.0,9777.0
上坑,2.0,294.0,1.0,1.0,0.0,0.0,39.0,2.0,8.0,NaN,...,4.0,2.0,2.0,0.0,1.0,1.0,22.0,0.0,1.0,5517.0
上岭排,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0
上排,1.0,107.0,16.0,6.0,1.0,27.0,3.0,36.0,9.0,4.0,...,2.0,2.0,2.0,2.0,1.0,2.0,4.0,1.0,1.0,9868.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
龙尾,0.0,10.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,2529.0
龙田,2.0,360.0,0.0,1.0,4.0,1.0,3.0,0.0,9.0,1.0,...,11.0,11.0,NaN,0.0,24.0,1.0,3.0,2.0,2.0,6632.0
龙眼山,5.0,278.0,14.0,1.0,0.0,1.0,1.0,16.0,25.0,2.0,...,0.0,1.0,0.0,NaN,0.0,0.0,13.0,0.0,2.0,3291.0


compared to...

In [28]:
cdatasubmat1

D_name,万丰,万科城,三合,三围,三溪,三祝里,三联,上下屋,上合,上坑,...,龙新,龙湖,龙田,龙眼山,龙红格,龙联,龙胜,龙西,龙辉,All
O_name,,,,,,,,,,,,,,,,,,,,,
三溪,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,2564.0
三联,0.0,457.0,8.0,0.0,0.0,0.0,NaN,0.0,10.0,12.0,...,4.0,25.0,1.0,0.0,4.0,5.0,25.0,0.0,19.0,9782.0
上坑,0.0,609.0,7.0,0.0,0.0,0.0,23.0,0.0,8.0,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,19.0,0.0,14.0,5545.0
上岭排,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0
上排,3.0,53.0,7.0,2.0,0.0,119.0,0.0,163.0,47.0,0.0,...,0.0,0.0,0.0,42.0,0.0,5.0,1.0,0.0,45.0,9872.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
龙尾,0.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,2565.0
龙田,0.0,2.0,0.0,0.0,1.0,0.0,25.0,0.0,2.0,2.0,...,102.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,6646.0
龙眼山,0.0,5.0,10.0,0.0,0.0,14.0,0.0,34.0,30.0,12.0,...,0.0,0.0,0.0,NaN,0.0,0.0,21.0,0.0,0.0,3312.0


and we can test the goodness-of-fit ine xactly the same was as before:

In [12]:
CalcRSqaured(cdatasub1["number"],cdatasub1["productionfitted"])

0.6339183766931376

In [13]:

CalcRMSE(cdatasub1["number"],cdatasub1["productionfitted"])

112.737

# Add more explanatory factor

In [3]:
# Cij

contiguity = pd.read_csv('for_luti/contiguity.csv')
contiguity

,ID,id2,contiguity
0,1,V1,0
1,1,V2,1
2,1,V3,0
3,1,V4,1
4,1,V5,0
...,...,...,...
609956,781,V777,0
609957,781,V778,0
609958,781,V779,0
609959,781,V780,1


In [4]:
#show the actual flows between boroughs
contiguitymat = pd.pivot_table(contiguity, values ="contiguity", index="ID", columns = "id2",
                            aggfunc=np.sum, margins=True)
#show the data
contiguitymat

id2,V1,V10,V100,V101,V102,V103,V104,V105,V106,V107,...,V91,V92,V93,V94,V95,V96,V97,V98,V99,All
ID,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
4,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
5,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
779,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
780,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [80]:
mobilityC1 = pd.read_csv("for_luti/mobilityC1.csv", index_col=0)
mobilityC1

,repaired_avg_time,Dest,Orig,number,dis,avg_time,ave_spd,population,job,log_population,log_job,log_repaired_avg_time,O_name,D_name
0,23.635000,3,2,2.0,1.12,23.635000,0.047387,2614,1959,7.868637,7.580189,3.162729,人民桥,红岭
1,3.966250,4,2,8.0,1.12,3.966250,0.282383,2614,6432,7.868637,8.769041,1.377821,人民桥,红南
2,4.013636,6,2,22.0,1.12,4.013636,0.279049,2614,27123,7.868637,10.208137,1.389698,人民桥,滨苑
3,8.991667,8,2,12.0,1.00,8.991667,0.111214,2614,14639,7.868637,9.591444,2.196298,人民桥,桂园老围
4,7.881379,9,2,29.0,1.41,7.881379,0.178903,2614,7706,7.868637,8.949755,2.064503,人民桥,松园
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73723,63.827851,275,778,0.0,0.00,0.000000,0.000000,9912,3366,9.201501,8.121480,4.156190,红棉,大铲
73724,61.378363,271,778,0.0,0.00,0.000000,0.000000,9912,957,9.201501,6.863803,4.117057,红棉,渔一
73725,68.535463,278,778,0.0,0.00,0.000000,0.000000,9912,710,9.201501,6.565265,4.227351,红棉,南海玫瑰园
73726,37.426618,663,778,0.0,0.00,0.000000,0.000000,9912,424,9.201501,6.049733,3.622382,红棉,西涌


In [81]:
mobilityC1['contiguity'] = 0
i = 0
for row in mobilityC1.iterrows():
    currnt_o = row[1]['Orig']
    currnt_d = row[1]['Dest']
    for_cmat_r = currnt_o + 1
    for_cmat_c = 'V'+ str(currnt_d + 1)
    c_value = contiguitymat.loc[for_cmat_r,for_cmat_c]
    if c_value == 1:
        mobilityC1.loc[i,'contiguity'] = 1
    i = i + 1

mobilityC1
    

,repaired_avg_time,Dest,Orig,number,dis,avg_time,ave_spd,population,job,log_population,log_job,log_repaired_avg_time,O_name,D_name,contiguity
0,23.635000,3,2,2.0,1.12,23.635000,0.047387,2614,1959,7.868637,7.580189,3.162729,人民桥,红岭,0
1,3.966250,4,2,8.0,1.12,3.966250,0.282383,2614,6432,7.868637,8.769041,1.377821,人民桥,红南,0
2,4.013636,6,2,22.0,1.12,4.013636,0.279049,2614,27123,7.868637,10.208137,1.389698,人民桥,滨苑,1
3,8.991667,8,2,12.0,1.00,8.991667,0.111214,2614,14639,7.868637,9.591444,2.196298,人民桥,桂园老围,0
4,7.881379,9,2,29.0,1.41,7.881379,0.178903,2614,7706,7.868637,8.949755,2.064503,人民桥,松园,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73723,63.827851,275,778,0.0,0.00,0.000000,0.000000,9912,3366,9.201501,8.121480,4.156190,红棉,大铲,0
73724,61.378363,271,778,0.0,0.00,0.000000,0.000000,9912,957,9.201501,6.863803,4.117057,红棉,渔一,0
73725,68.535463,278,778,0.0,0.00,0.000000,0.000000,9912,710,9.201501,6.565265,4.227351,红棉,南海玫瑰园,0
73726,37.426618,663,778,0.0,0.00,0.000000,0.000000,9912,424,9.201501,6.049733,3.622382,红棉,西涌,0


In [58]:
#create the formula (the "-1" indicates no intercept in the regression model).
singO_form = 'number ~ O_name + log_job + log_repaired_avg_time + contiguity -1'
#run a doubly constrained sim
singOSim = smf.glm(formula = singO_form, data=mobilityC1, family=sm.families.Poisson()).fit()


In [59]:
#let's have a look at it's summary
singOSim.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 number   No. Observations:               119808
Model:                            GLM   Df Residuals:                   119597
Model Family:                 Poisson   Df Model:                          210
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.4546e+06
Date:                Mon, 31 Jul 2023   Deviance:                   2.7413e+06
Time:                        16:27:11   Pearson chi2:                 9.74e+06
No. Iterations:                    12   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
O_name[三溪]               -0.7982      0.021    -38.890      0.000      -0.838      -0.758
O_name[三联]               -0.4106      0.012    -34.820      0.000      -0.434      -0.387
O_name[上坑]               -1.6293      0.015   -109.141      0.000      -1.659      -1.600
O_name[上岭排]              -7.9138      0.180    -44.036      0.000      -8.266      -7.562
O_name[上排]               -0.3139      0.012    -26.766      0.000      -0.337      -0.291
O_name[上木古]              -2.5393      0.023   -108.676      0.000      -2.585      -2.493
O_name[上村]               -0.5784      0.014    -41.027      0.000      -0.606      -0.551
O_name[下岭排]              -2.1096      0.022    -95.363      0.000      -2.153      -2.066
O_name[下排]               -3.7050      0.024   -155.064      0.000      -3.752      -3.658
O_name[下李朗]              -0.0881      0.011     -8.127      0.000      -0.109      -0.067
O_name[下村]                1.0804      0.009    124.245      0.000       1.063       1.097
O_name[下横朗]              -0.8082      0.014    -55.871      0.000      -0.837      -0.780
O_name[下湖]               -0.0426      0.011     -3.956      0.000      -0.064      -0.021
O_name[东山]               -1.5561      0.050    -31.197      0.000      -1.654      -1.458
O_name[东方半岛]             -0.2263      0.011    -21.301      0.000      -0.247      -0.205
O_name[东方社区二]             0.1366      0.011     12.999      0.000       0.116       0.157
O_name[东海岸]              -1.2174      0.020    -61.853      0.000      -1.256      -1.179
O_name[东涌]               -3.2543      0.131    -24.753      0.000      -3.512      -2.997
O_name[书院街]              -0.3929      0.010    -39.231      0.000      -0.413      -0.373
O_name[五湾]               -2.0572      0.020   -105.198      0.000      -2.096      -2.019
O_name[五联]                0.8214      0.010     80.369      0.000       0.801       0.841
O_name[仙湖]               -1.8254      0.022    -82.066      0.000      -1.869      -1.782
O_name[保安]               -0.2127      0.012    -18.396      0.000      -0.235      -0.190
O_name[倚山]               -1.6462      0.021    -77.953      0.000      -1.688      -1.605
O_name[元芬]               -1.7709      0.018    -97.568      0.000      -1.806      -1.735
O_name[光明]               -6.9255      0.243    -28.546      0.000      -7.401      -6.450
O_name[光明凤凰]             -5.0601      0.086    -58.873      0.000      -5.229      -4.892
O_name[公明红星]             -2.9121      0.033    -87.561      0.000      -2.977      -2.847
O_name[兰花]               -0.3329      0.010    -32.246      0.000      -0.353      -0.313
O_name[凤凰岗]              -3.6290      0.027   -132.079      0.000      -3.683      -3.575
O_name[南坑]                0.8542      0.008    109.155      0.000       0.839       0.870
O_name[南头城]            

In [60]:
#get the estimates
mobilityC1["originsimfittedwithC"] = np.round(singOSim.mu)
#here's the matrix
cdatasubmat = mobilityC1.pivot_table(values ="originsimfittedwithC", index="O_name", columns = "D_name",
                                    aggfunc=np.sum, margins=True)
cdatasubmat

D_name,万丰,万科城,三合,三围,三溪,三祝里,三联,上下屋,上合,上坑,...,龙新,龙湖,龙田,龙眼山,龙红格,龙联,龙胜,龙西,龙辉,All
O_name,,,,,,,,,,,,,,,,,,,,,
三溪,3.0,37.0,0.0,1.0,NaN,0.0,1.0,0.0,2.0,1.0,...,6.0,0.0,20.0,0.0,1.0,1.0,2.0,1.0,1.0,2533.0
三联,18.0,357.0,1.0,1.0,2.0,1.0,NaN,0.0,5.0,23.0,...,2.0,9.0,7.0,0.0,17.0,4.0,13.0,1.0,7.0,9776.0
上坑,2.0,189.0,1.0,1.0,0.0,0.0,25.0,2.0,7.0,NaN,...,3.0,1.0,2.0,0.0,1.0,1.0,16.0,0.0,1.0,5512.0
上岭排,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0
上排,2.0,110.0,15.0,7.0,2.0,23.0,4.0,91.0,11.0,5.0,...,3.0,3.0,3.0,2.0,1.0,3.0,5.0,1.0,2.0,9871.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
龙尾,1.0,13.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,3.0,1.0,0.0,2.0,2530.0
龙田,4.0,305.0,0.0,1.0,4.0,1.0,4.0,1.0,11.0,1.0,...,11.0,11.0,NaN,0.0,21.0,1.0,4.0,2.0,3.0,6636.0
龙眼山,6.0,213.0,11.0,2.0,0.0,1.0,1.0,12.0,23.0,2.0,...,0.0,1.0,0.0,NaN,0.0,1.0,12.0,0.0,2.0,3293.0


In [61]:
cdatasubmat = mobilityC1.pivot_table(values ="number", index="O_name", columns = "D_name",
                                    aggfunc=np.sum, margins=True)
cdatasubmat

D_name,万丰,万科城,三合,三围,三溪,三祝里,三联,上下屋,上合,上坑,...,龙新,龙湖,龙田,龙眼山,龙红格,龙联,龙胜,龙西,龙辉,All
O_name,,,,,,,,,,,,,,,,,,,,,
三溪,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,2564.0
三联,0.0,457.0,8.0,0.0,0.0,0.0,NaN,0.0,10.0,12.0,...,4.0,25.0,1.0,0.0,4.0,5.0,25.0,0.0,19.0,9782.0
上坑,0.0,609.0,7.0,0.0,0.0,0.0,23.0,0.0,8.0,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,19.0,0.0,14.0,5545.0
上岭排,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0
上排,3.0,53.0,7.0,2.0,0.0,119.0,0.0,163.0,47.0,0.0,...,0.0,0.0,0.0,42.0,0.0,5.0,1.0,0.0,45.0,9872.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
龙尾,0.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,2565.0
龙田,0.0,2.0,0.0,0.0,1.0,0.0,25.0,0.0,2.0,2.0,...,102.0,0.0,NaN,0.0,1.0,0.0,0.0,0.0,0.0,6646.0
龙眼山,0.0,5.0,10.0,0.0,0.0,14.0,0.0,34.0,30.0,12.0,...,0.0,0.0,0.0,NaN,0.0,0.0,21.0,0.0,0.0,3312.0


In [9]:
CalcRSqaured(mobilityC1["number"],mobilityC1["originsimfittedwithC"])


NameError: name 'mobilityC1' is not defined

In [63]:
CalcRMSE(mobilityC1["number"],mobilityC1["originsimfittedwithC"])

84.902

# add single POI

In [7]:
POI2018 = pd.read_csv(r'data/poi2018/szpoi2018_wgs/count_poi2018sz.csv')
POI2018 = POI2018.drop(['Unnamed: 0'],axis =1)
POI2018

,id,catering,edu,finance,firm,gov,health,hotel,landscape,living,public,residence,shopping,sports,traffic,vehicle
0,0,22,24,29,86,32,4,31,6,28,2,20,25,9,46,0
1,1,28,40,6,46,23,5,19,0,35,4,31,27,15,33,1
2,2,116,70,102,160,26,31,189,9,167,61,66,644,34,132,1
3,3,110,39,20,15,19,32,50,2,93,5,26,121,11,62,1
4,4,88,52,46,51,108,27,158,2,127,8,46,161,18,123,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,776,8,33,4,3,5,2,0,6,9,4,24,4,7,26,0
777,777,154,85,28,87,40,38,7,10,89,11,88,173,46,73,8
778,778,29,43,9,18,14,13,0,2,40,0,28,53,13,44,2
779,779,18,6,6,3,4,11,4,0,22,0,4,34,10,15,1


In [82]:
mobilityC1['d_finance'] = 0
mobilityC1['d_traffic'] = 0
mobilityC1['d_firm'] = 0
mobilityC1['d_edu'] = 0
mobilityC1['d_residence'] = 0
mobilityC1['d_public'] = 0
i = 0
for row in mobilityC1.iterrows():
    currnt_d = row[1]['Dest']
    finance = POI2018.loc[currnt_d,'finance']
    traffic = POI2018.loc[currnt_d,'traffic']
    firm = POI2018.loc[currnt_d,'firm']
    edu = POI2018.loc[currnt_d,'edu']
    residence = POI2018.loc[currnt_d,'residence']
    public = POI2018.loc[currnt_d,'public']
    mobilityC1.loc[i,'d_finance'] = finance
    mobilityC1.loc[i,'d_traffic'] = traffic
    mobilityC1.loc[i,'d_firm'] = firm
    mobilityC1.loc[i,'d_edu'] = edu
    mobilityC1.loc[i,'d_residence'] = residence
    mobilityC1.loc[i,'d_public'] = public   
    i = i + 1

mobilityC1
    

,repaired_avg_time,Dest,Orig,number,dis,avg_time,ave_spd,population,job,log_population,...,log_repaired_avg_time,O_name,D_name,contiguity,d_finance,d_traffic,d_firm,d_edu,d_residence,d_public
0,23.635000,3,2,2.0,1.12,23.635000,0.047387,2614,1959,7.868637,...,3.162729,人民桥,红岭,0,20,62,15,39,26,5
1,3.966250,4,2,8.0,1.12,3.966250,0.282383,2614,6432,7.868637,...,1.377821,人民桥,红南,0,46,123,51,52,46,8
2,4.013636,6,2,22.0,1.12,4.013636,0.279049,2614,27123,7.868637,...,1.389698,人民桥,滨苑,1,17,97,11,19,30,9
3,8.991667,8,2,12.0,1.00,8.991667,0.111214,2614,14639,7.868637,...,2.196298,人民桥,桂园老围,0,153,108,76,34,47,22
4,7.881379,9,2,29.0,1.41,7.881379,0.178903,2614,7706,7.868637,...,2.064503,人民桥,松园,0,95,210,312,163,80,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73723,63.827851,275,778,0.0,0.00,0.000000,0.000000,9912,3366,9.201501,...,4.156190,红棉,大铲,0,14,22,5,18,20,8
73724,61.378363,271,778,0.0,0.00,0.000000,0.000000,9912,957,9.201501,...,4.117057,红棉,渔一,0,11,37,23,15,29,4
73725,68.535463,278,778,0.0,0.00,0.000000,0.000000,9912,710,9.201501,...,4.227351,红棉,南海玫瑰园,0,3,8,2,10,7,0
73726,37.426618,663,778,0.0,0.00,0.000000,0.000000,9912,424,9.201501,...,3.622382,红棉,西涌,0,4,42,2,6,6,9


In [10]:

#take the variables and produce logarithms of them
x_variables = ["d_finance"]
log_x_vars = []
for x in x_variables:
    mobilityC1[f"log_{x}"] = np.log(mobilityC1[x])
    log_x_vars.append(f"log_{x}")
mobilityC1

/Users/wuyuyang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,repaired_avg_time,Dest,Orig,number,dis,avg_time,ave_spd,population,job,log_population,log_job,log_repaired_avg_time,O_name,D_name,contiguity,d_finance,log_d_finance
0,23.635000,3,2,2.0,1.12,23.635000,0.047387,2614,1959,7.868637,7.580189,3.162729,人民桥,红岭,0,20,2.995732
1,3.966250,4,2,8.0,1.12,3.966250,0.282383,2614,6432,7.868637,8.769041,1.377821,人民桥,红南,0,46,3.828641
2,4.013636,6,2,22.0,1.12,4.013636,0.279049,2614,27123,7.868637,10.208137,1.389698,人民桥,滨苑,1,17,2.833213
3,8.991667,8,2,12.0,1.00,8.991667,0.111214,2614,14639,7.868637,9.591444,2.196298,人民桥,桂园老围,0,153,5.030438
4,7.881379,9,2,29.0,1.41,7.881379,0.178903,2614,7706,7.868637,8.949755,2.064503,人民桥,松园,0,95,4.553877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73723,63.827851,275,778,0.0,0.00,0.000000,0.000000,9912,3366,9.201501,8.121480,4.156190,红棉,大铲,0,14,2.639057
73724,61.378363,271,778,0.0,0.00,0.000000,0.000000,9912,957,9.201501,6.863803,4.117057,红棉,渔一,0,11,2.397895
73725,68.535463,278,778,0.0,0.00,0.000000,0.000000,9912,710,9.201501,6.565265,4.227351,红棉,南海玫瑰园,0,3,1.098612
73726,37.426618,663,778,0.0,0.00,0.000000,0.000000,9912,424,9.201501,6.049733,3.622382,红棉,西涌,0,4,1.386294


In [83]:
#create the formula (the "-1" indicates no intercept in the regression model).
#+ d_edu + d_residence + d_public
singO_form = 'number ~ O_name + log_job + log_repaired_avg_time + contiguity + d_finance + d_traffic + d_firm + d_edu + d_residence + d_public -1'
#run a doubly constrained sim
singOSim = smf.glm(formula = singO_form, data=mobilityC1, family=sm.families.Poisson()).fit()

In [84]:
#let's have a look at it's summary
singOSim.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 number   No. Observations:                73728
Model:                            GLM   Df Residuals:                    73591
Model Family:                 Poisson   Df Model:                          136
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -8.7743e+05
Date:                Tue, 01 Aug 2023   Deviance:                   1.6099e+06
Time:                        16:24:41   Pearson chi2:                 2.98e+06
No. Iterations:                     8   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
O_name[东方社区]             -2.4709      0.011   -223.372      0.000      -2.493      -2.449
O_name[东民]               -3.0847      0.014   -228.493      0.000      -3.111      -3.058
O_name[东海社区]             -4.0170      0.016   -245.018      0.000      -4.049      -3.985
O_name[东湖乐群]             -4.2666      0.020   -212.987      0.000      -4.306      -4.227
O_name[东角头]              -2.6783      0.012   -225.605      0.000      -2.702      -2.655
O_name[东门]               -4.4468      0.020   -219.191      0.000      -4.487      -4.407
O_name[乐群社区]             -1.0554      0.010   -106.184      0.000      -1.075      -1.036
O_name[京光]               -3.3702      0.013   -266.481      0.000      -3.395      -3.345
O_name[人民桥]              -4.4574      0.021   -208.374      0.000      -4.499      -4.415
O_name[光华]               -3.7483      0.018   -208.399      0.000      -3.784      -3.713
O_name[前海]               -4.2758      0.018   -242.988      0.000      -4.310      -4.241
O_name[华乐]               -2.5540      0.014   -180.533      0.000      -2.582      -2.526
O_name[华红]               -5.2257      0.024   -220.677      0.000      -5.272      -5.179
O_name[华航]               -3.5515      0.014   -245.089      0.000      -3.580      -3.523
O_name[南光]               -2.3046      0.011   -219.444      0.000      -2.325      -2.284
O_name[南油]               -3.6381      0.014   -263.440      0.000      -3.665      -3.611
O_name[南海玫瑰园]            -5.1422      0.028   -182.850      0.000      -5.197      -5.087
O_name[可园]               -2.3834      0.012   -193.510      0.000      -2.408      -2.359
O_name[后海]               -2.9218      0.012   -251.237      0.000      -2.945      -2.899
O_name[向南]               -2.7343      0.011   -242.822      0.000      -2.756      -2.712
O_name[嘉北]               -3.9589      0.016   -245.258      0.000      -3.991      -3.927
O_name[嘉南]               -3.3420      0.015   -224.357      0.000      -3.371      -3.313
O_name[四季花城]             -1.4186      0.010   -140.251      0.000      -1.438      -1.399
O_name[四海]               -2.8950      0.012   -244.805      0.000      -2.918      -2.872
O_name[园东]               -4.0045      0.017   -240.033      0.000      -4.037      -3.972
O_name[园西]               -4.0549      0.017   -242.394      0.000      -4.088      -4.022
O_name[围仔]               -2.2085      0.011   -201.329      0.000      -2.230      -2.187
O_name[大冲]               -4.5780      0.019   -237.304      0.000      -4.616      -4.540
O_name[大坪]               -4.5616      0.029   -155.907      0.000      -4.619      -4.504
O_name[大学城]              -1.2957      0.010   -129.026      0.000      -1.315      -1.276
O_name[大新]               -2.5907      0.011   -235.594      0.000      -2.612      -2.569
O_name[大铲]             

In [85]:
mobilityC1["originsimfittedwithC_f_t_f_e_r_d"] = np.round(singOSim.mu)
#_e_r_d

In [86]:
CalcRSqaured(mobilityC1["number"],mobilityC1["originsimfittedwithC_f_t_f_e_r_d"])

0.7274797983008707

In [87]:
CalcRMSE(mobilityC1["number"],mobilityC1["originsimfittedwithC_f_t_f_e_r_d"])

77.804

In [88]:
mobilityC1.to_csv("for_luti/mobilityC1_withoutloop_forLUTI_withname_c_6poi.csv")

In [69]:
mobilityC1

,repaired_avg_time,Dest,Orig,number,dis,avg_time,ave_spd,population,job,log_population,...,D_name,contiguity,d_finance,d_traffic,d_firm,d_edu,d_residence,d_public,originsimfittedwithC_f_t_f_e_r_d,originsimfittedwithC_f_t_f
0,23.983714,2,15,35.0,3.886571,23.983714,0.162050,3150,19106,8.055158,...,人民桥,0,102,132,160,70,66,61,6.0,6.0
1,4.150000,3,15,1.0,4.030000,4.150000,0.971084,3150,1959,8.055158,...,红岭,0,20,62,15,39,26,5,18.0,18.0
2,6.551818,4,15,11.0,5.100000,6.551818,0.778410,3150,6432,8.055158,...,红南,0,46,123,51,52,46,8,23.0,24.0
3,10.601579,6,15,19.0,4.920000,10.601579,0.464082,3150,27123,8.055158,...,滨苑,0,17,97,11,19,30,9,39.0,38.0
4,9.487727,8,15,22.0,5.105455,9.487727,0.538111,3150,14639,8.055158,...,桂园老围,0,153,108,76,34,47,22,26.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119803,42.179550,659,777,0.0,0.000000,0.000000,0.000000,43950,801,10.690808,...,南隆,0,14,32,9,24,50,3,1.0,1.0
119804,41.157162,662,777,0.0,0.000000,0.000000,0.000000,43950,290,10.690808,...,东涌,0,3,12,0,2,1,4,1.0,1.0
119805,45.777938,248,777,0.0,0.000000,0.000000,0.000000,43950,93,10.690808,...,月亮湾,0,4,80,22,15,30,4,0.0,0.0
119806,32.029764,278,777,0.0,0.000000,0.000000,0.000000,43950,710,10.690808,...,南海玫瑰园,0,3,8,2,10,7,0,2.0,2.0
